# Start Dask Cluster

In [32]:
from dask.distributed import Client
from dask.distributed import LocalCluster
import os

In [33]:
os.system(f"rm -rf ./dask-worker-space")

0

In [34]:
cluster = LocalCluster(silence_logs=50)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 48,Total memory: 125.65 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40737,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 48
Started: Just now,Total memory: 125.65 GiB
Comm: tcp://127.0.0.1:35419,Total threads: 6
Dashboard: http://127.0.0.1:44529/status,Memory: 15.71 GiB
Nanny: tcp://127.0.0.1:42877,


# Import other libraries

In [35]:
import dask.dataframe as pd
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans, DBSCAN
from sklearn.ensemble import RandomForestClassifier
from dask_ml.xgboost import XGBClassifier
from xgboost.dask import DaskDMatrix
import xgboost as xgb
from sklearn.decomposition import TruncatedSVD, PCA
import numpy as np
from dask_ml.preprocessing import LabelEncoder
import joblib

# Input filename
Adjust the filename to one of the files available in the sample_data folder.  
Files are availble publicly for download

In [36]:
in_filename = 'sample_data_20m.csv'

# Define variables

In [37]:
label_col_name = 'is_counterfeit' # the name of the column that contain the labels (0 for negative, 1 for positive)

predictor_col_names = ['Size of U.S. market', # the names of the columns that will be used to predict the label
                       'Price per unit', 
                       'RX/OTC', 
                       'Indication',
                       'Drug Class',
                       'Shortage',
                       'Twitter Mentions'
                      ] 

cetegoric_predictor_col_names = ['RX/OTC', # list of predictor columns that are not values; will use a label encoder on these
                                 'Indication',
                                 'Drug Class', 
                                 'Shortage'
                                ]

# models perform better when values are between 0 and 1. 
# thus, we use a scaling value to attempt to do this
scaling_dict = {'Size of U.S. market': 1/100000000000, # each column in this dictionary will be multiplied by the values here
                'Price per unit': 1/10000,
                'Twitter Mentions': 1/200000
               }

val_set_percent = 0.1 # percent of data to be reserved for validation
test_set_percent = 0.1 # percent of data to be reserved for testing

chunksize = "100MB"
blocksize = "100MB"

random_seed = 42 # to make results reproducible

In [38]:
out_filename = 'write_speed_test_gpu.csv'
in_filename = './sample_data/' + in_filename
out_filename = './output_data/' + out_filename

# Define class for temp file names

In [39]:
class TempFileNameSwitcher:
    def __init__(self):
        self.temp_names = ['./output_data/temp_1.csv', './output_data/temp_2.csv']
        self.cur_iter = 0
        self.name = self.temp_names[self.cur_iter]
        for a_name in self.temp_names:
            try:
                os.system(f"rm -rf {a_name}")
            except:
                pass
            try:
                os.system(f"rm -f {a_name}")
            except:
                pass
        
    def switch(self):
        self.cur_iter += 1
        name_i = self.cur_iter % len(self.temp_names)
        self.name = self.temp_names[name_i]

In [40]:
temp_file_name_switcher = TempFileNameSwitcher()

# ETL

## Single CSV file

### read

In [41]:
df = pd.read_csv(in_filename, blocksize=blocksize)

In [42]:
%%time
_ = df.compute()

CPU times: user 4.56 s, sys: 3.07 s, total: 7.63 s
Wall time: 10.8 s


### write

In [43]:
os.system(f"rm -rf {out_filename}")
os.system(f"rm -f {out_filename}")

0

In [44]:
%%time
df.to_csv(out_filename, index=False, single_file=True)

CPU times: user 7.5 s, sys: 1.73 s, total: 9.23 s
Wall time: 1min 16s


['/home/jcosme/proj/GPU_vs_CPU_v2/output_data/write_speed_test_gpu.csv']

## Multiple CSV files

In [45]:
os.system(f"rm -rf {out_filename}")
os.system(f"rm -f {out_filename}")

0

### write

In [46]:
%%time
df.to_csv(out_filename, index=False)

CPU times: user 2.09 s, sys: 475 ms, total: 2.56 s
Wall time: 18.7 s


['/home/jcosme/proj/GPU_vs_CPU_v2/output_data/write_speed_test_gpu.csv/00.part',
 '/home/jcosme/proj/GPU_vs_CPU_v2/output_data/write_speed_test_gpu.csv/01.part',
 '/home/jcosme/proj/GPU_vs_CPU_v2/output_data/write_speed_test_gpu.csv/02.part',
 '/home/jcosme/proj/GPU_vs_CPU_v2/output_data/write_speed_test_gpu.csv/03.part',
 '/home/jcosme/proj/GPU_vs_CPU_v2/output_data/write_speed_test_gpu.csv/04.part',
 '/home/jcosme/proj/GPU_vs_CPU_v2/output_data/write_speed_test_gpu.csv/05.part',
 '/home/jcosme/proj/GPU_vs_CPU_v2/output_data/write_speed_test_gpu.csv/06.part',
 '/home/jcosme/proj/GPU_vs_CPU_v2/output_data/write_speed_test_gpu.csv/07.part',
 '/home/jcosme/proj/GPU_vs_CPU_v2/output_data/write_speed_test_gpu.csv/08.part',
 '/home/jcosme/proj/GPU_vs_CPU_v2/output_data/write_speed_test_gpu.csv/09.part',
 '/home/jcosme/proj/GPU_vs_CPU_v2/output_data/write_speed_test_gpu.csv/10.part',
 '/home/jcosme/proj/GPU_vs_CPU_v2/output_data/write_speed_test_gpu.csv/11.part',
 '/home/jcosme/proj/GPU_vs_C

### read

In [47]:
df = pd.read_csv(f"{out_filename}/*", blocksize=blocksize)

In [48]:
%%time
_ = df.compute()

CPU times: user 4.71 s, sys: 3.16 s, total: 7.87 s
Wall time: 10.8 s


## Common DataFrame Operations

In [49]:
df = pd.read_csv(in_filename, blocksize=blocksize)

### describe the dataframe

In [50]:
%%time
df.describe().compute()

CPU times: user 1.05 s, sys: 202 ms, total: 1.25 s
Wall time: 3.76 s


,is_counterfeit,Size of U.S. market,Price per unit,Twitter Mentions,SNOMED
count,2.000000e+07,2.000000e+07,2.000000e+07,2.000000e+07,2.000000e+07
mean,3.800000e-01,2.319031e+09,2.073848e+02,9.099047e+03,2.584983e+08
std,4.853865e-01,3.181265e+09,3.072970e+02,5.702964e+03,2.758066e+08
min,0.000000e+00,1.002100e+04,1.000000e-01,0.000000e+00,7.321101e+07
25%,0.000000e+00,4.998876e+07,2.540000e+00,6.249000e+03,8.065901e+07
50%,1.000000e+00,1.743000e+09,6.750000e+01,1.061800e+04,1.192920e+08
75%,1.000000e+00,7.259273e+09,6.904100e+02,1.688200e+04,3.736210e+08
max,1.000000e+00,9.999998e+09,1.000000e+03,2.000000e+04,8.403580e+08


### Set Index
for each categorical variable, set the DataFrame index to that variable
  
**note:** dask dataframes have a 'set_index' method, but it is slower than applying the 'set_index' method to each partition (so we will use the latter method).

In [51]:
def set_cat_index(df, a_cat):
    return df.set_index(a_cat)

In [52]:
%%time
for a_cat in cetegoric_predictor_col_names:
    print(a_cat)
    _ = df.map_partitions(set_cat_index, a_cat).compute()

RX/OTC
Indication
Drug Class
Shortage
CPU times: user 17.7 s, sys: 10.8 s, total: 28.5 s
Wall time: 41.1 s


### Concat multiple DataFrames
concatenate 3 copies of the DF

In [53]:
%%time
_ = pd.concat([df] * 3, ignore_index=True).compute()

CPU times: user 12.7 s, sys: 8.78 s, total: 21.5 s
Wall time: 26.1 s


### Groupby function
for each value column, we calculate the mean by grouping by each categorical variable

In [54]:
non_cetegoric_predictor_col_names = list(set(predictor_col_names).difference(set(cetegoric_predictor_col_names)))
non_cetegoric_predictor_col_names

['Twitter Mentions', 'Price per unit', 'Size of U.S. market']

In [55]:
%%time
for a_cat in cetegoric_predictor_col_names:
    print(a_cat)
    for a_col in non_cetegoric_predictor_col_names:
        print(f"\t{a_col}")
        _ = df.groupby(a_cat)[a_col].mean().compute()

RX/OTC
	Twitter Mentions
	Price per unit
	Size of U.S. market
Indication
	Twitter Mentions
	Price per unit
	Size of U.S. market
Drug Class
	Twitter Mentions
	Price per unit
	Size of U.S. market
Shortage
	Twitter Mentions
	Price per unit
	Size of U.S. market
CPU times: user 7.07 s, sys: 1.49 s, total: 8.56 s
Wall time: 37.4 s


## Preprocess data

In [56]:
df_input = df.copy()

### fit label encoder
first we create a label encoder for each column specified in the variable 'cetegoric_predictor_col_names' 
defined at the top of the notebook

In [57]:
%%time
cats_dict = {}
for col in df_input.columns:
    if col in cetegoric_predictor_col_names:
        print(col)
        cats_dict[col] = LabelEncoder()
        cats_dict[col].fit(df_input[col])

RX/OTC
Indication
Shortage
Drug Class
CPU times: user 3.04 s, sys: 615 ms, total: 3.66 s
Wall time: 19.8 s


### encode categoric columns
Then we apply the label encoder to the values of those columns

In [58]:
%%time
for col in df_input.columns:
    if col in cetegoric_predictor_col_names:
        print(col)
        df_input[col] = cats_dict[col].transform(df_input[col])

RX/OTC
Indication
Shortage
Drug Class
CPU times: user 2.18 s, sys: 499 ms, total: 2.68 s
Wall time: 13.5 s


we save a temporary file, and reload the data from this file, so we can isolate the computation time of the next transformation (scale value variable columns).  
Notice we do NOT save the csv file as a 'single file.' This operation is faster, but it means that we will have multiple files, instead of one.

In [59]:
%%time
old_name = temp_file_name_switcher.name
temp_file_name_switcher.switch()
_ = df_input.to_csv(temp_file_name_switcher.name, index=False)
df_input = pd.read_csv(f"{temp_file_name_switcher.name}/*")
os.system(f"rm -rf {old_name}")

CPU times: user 2.45 s, sys: 569 ms, total: 3.02 s
Wall time: 19.2 s


0

### scale value variable columns
Next we apply the scaling by the variable 'scaling_dict' defined at the top of the notebook

In [60]:
%%time
for col in scaling_dict.keys():
    df_input[col] = df_input[col] * scaling_dict[col]
_ = df_input.compute()

CPU times: user 2.8 s, sys: 2.57 s, total: 5.37 s
Wall time: 7.76 s


we save a temporary file, and reload the data from this file, so we can isolate the computation time of the next transformation (train/val/test split).  
Notice we do NOT save the csv file as a 'single file.' This operation is faster, but it means that we will have multiple files, instead of one.

In [61]:
%%time
old_name = temp_file_name_switcher.name
temp_file_name_switcher.switch()
_ = df_input.to_csv(temp_file_name_switcher.name, index=False)
df_input = pd.read_csv(f"{temp_file_name_switcher.name}/*")
os.system(f"rm -rf {old_name}")

CPU times: user 2.41 s, sys: 529 ms, total: 2.94 s
Wall time: 18.1 s


0

## train/val/test split

In [62]:
not_train_prct = val_set_percent + test_set_percent
train_prct = 1. - not_train_prct

In [63]:
df_train, df_val, df_test = df_input.random_split([train_prct, val_set_percent, test_set_percent], random_state=random_seed)

In [64]:
%%time
_ = df_train.compute()
_ = df_val.compute()
_ = df_test.compute()

CPU times: user 4.41 s, sys: 2.77 s, total: 7.18 s
Wall time: 13.3 s


In [65]:
X_train = df_train[predictor_col_names].copy().astype(float)
y_train = df_train[label_col_name].copy().astype(float)

X_val = df_val[predictor_col_names].copy().astype(float)
y_val = df_val[label_col_name].copy().astype(float)

X_test = df_test[predictor_col_names].copy().astype(float)
y_test = df_test[label_col_name].copy().astype(float)

we save temporary files for each split, and reload them from this file, so we can isolate the computation time of the next operations (dimensionality reduction & model fitting).

In [66]:
os.system(f"rm -rf ./output_data/X_train")
os.system(f"rm -rf ./output_data/y_train")

os.system(f"rm -rf ./output_data/X_val")
os.system(f"rm -rf ./output_data/y_val")

os.system(f"rm -rf ./output_data/X_test")
os.system(f"rm -rf ./output_data/y_test")

0

In [67]:
%%time
_ = X_train.to_csv('./output_data/X_train', index=False)
_ = y_train.to_frame().to_csv('./output_data/y_train', index=False)

_ = X_val.to_csv('./output_data/X_val', index=False)
_ = y_val.to_frame().to_csv('./output_data/y_val', index=False)

_ = X_test.to_csv('./output_data/X_test', index=False)
_ = y_test.to_frame().to_csv('./output_data/y_test', index=False)

CPU times: user 5.52 s, sys: 1.11 s, total: 6.63 s
Wall time: 34.5 s


In [68]:
%%time
X_train = pd.read_csv('./output_data/X_train/*')
y_train = pd.read_csv('./output_data/y_train/*')
y_train = y_train[y_train.columns[0]]

X_val = pd.read_csv('./output_data/X_val/*')
y_val = pd.read_csv('./output_data/y_val/*')
y_val = y_val[y_val.columns[0]]

X_test = pd.read_csv('./output_data/X_test/*')
y_test = pd.read_csv('./output_data/y_test/*')
y_test = y_test[y_test.columns[0]]

CPU times: user 36.9 ms, sys: 3.78 ms, total: 40.7 ms
Wall time: 36.9 ms


In [69]:
del df, df_input

# Dimensionality Reduction (2D)

## PCA

In [70]:
%%time
with joblib.parallel_backend('dask'):
    pca = PCA(n_components=2, random_state=random_seed)
    # pca_incr = Incremental(pca)
    X_train_pca = pca.fit_transform(X_train)
    # pca.fit(X_train.compute())

CPU times: user 1min 23s, sys: 1min 9s, total: 2min 33s
Wall time: 14.8 s


In [71]:
del pca, X_train_pca

## TruncatedSVD

In [72]:
%%time
with joblib.parallel_backend('dask'):
    trunSVD = TruncatedSVD(n_components=2, random_state=random_seed)
    X_train_trunSVD = trunSVD.fit_transform(X_train)

CPU times: user 1min 35s, sys: 1min 19s, total: 2min 54s
Wall time: 14.8 s


In [73]:
del trunSVD, X_train_trunSVD

# Models

## OLS Regression

In [74]:
%%time
with joblib.parallel_backend('dask'):
    reg = LinearRegression()
    reg.fit(X_train, y_train)

CPU times: user 7.83 s, sys: 17.6 s, total: 25.4 s
Wall time: 5.17 s


In [75]:
del reg

## K-Means

In [76]:
n_clusters = y_train.unique().compute().shape[0]
n_clusters

2

In [77]:
%%time
with joblib.parallel_backend('dask'):
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_seed)
    kmeans.fit(X_train)

CPU times: user 1min 1s, sys: 1min, total: 2min 2s
Wall time: 24.1 s


In [78]:
del kmeans

## Gradient Boosting

In [79]:
%%time
dtrain = DaskDMatrix(client, X_train, y_train)

CPU times: user 184 ms, sys: 62.8 ms, total: 246 ms
Wall time: 763 ms


In [80]:
%%time
xgb_model = xgb.dask.train(
        client,
    {"tree_method": "hist", "random_state": random_seed},
    dtrain,
    
)


/home/jcosme/miniconda3/envs/gvc-2/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/jcosme/miniconda3/envs/gvc-2/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/jcosme/miniconda3/envs/gvc-2/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/jcosme/miniconda3/envs/gvc-2/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/jcosme/miniconda3/envs/gvc-2/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Work

CPU times: user 679 ms, sys: 205 ms, total: 884 ms
Wall time: 4.4 s


In [81]:
del xgb

In [82]:
client.shutdown()

In [83]:
del client